In [ ]:
# import packages

import os
from operator import itemgetter    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import funcsigs
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV,KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from funcsigs import signature
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from mlxtend.plotting import plot_learning_curves
from mlxtend.preprocessing import shuffle_arrays_unison

import tensorflow as tf
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import math
import seaborn as sns
from IPython.display import display

from statsmodels.formula import api
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [ ]:
# read in the data

df = pd.read_excel(r"dataset.xlsx")
print(df.head())


In [ ]:
# set target and features

target = 'COPD_Diagnose_ja_nein'
features = [i for i in df.columns if i not in [target]]

# make a copy of the original data

original_df = df.copy(deep=True)


In [ ]:
# show what are in the work space.

dir()


In [ ]:
# display some data information 

print(df.shape)


In [ ]:
# display some data information 

print(df.info())


In [ ]:
# display some data information 

print(df.describe())


In [ ]:
# check the number of values for all columns 

df.nunique().sort_values()


In [ ]:
# figure out numerical and categorical features

nu = df[features].nunique().sort_values()
nf = []; 
cf = []; 
nnf = 0; 
ncf = 0; 

for i in range(df[features].shape[1]):
    if nu.values[i]<=16: 
        cf.append(nu.index[i])
    else: nf.append(nu.index[i])

print('\n\033[1mInference:\033[0m The Datset has {} numerical and {} categorical features.'.format(len(nf),len(cf)))


In [ ]:
# set how many numerical features and categorical features.

Nnf = 13
Ncf = 6


In [ ]:
# count plot for categorical features

print('\033[1mVisualising Categorical Features:'.center(100))

fig, axs = plt.subplots(nrows = 2, ncols = 3)
ax = axs.flatten()
for i in range(len(cf)):
    if df[cf[i]].nunique()<=16:
        sns.countplot(x = cf[i], data = df, ax = ax[i])
        
plt.tight_layout()
plt.show()


In [ ]:
# distribution plot for numerical features

print('\033[1mNumeric Features Distribution'.center(130))

fig, axs = plt.subplots(nrows = 5, ncols = 3)
ax = axs.flatten()
for i in range(len(nf)):
    sns.distplot(df[nf[i]], ax = ax[i], hist_kws=dict(edgecolor="black", linewidth=2), bins=10)
plt.tight_layout()
ax[13].set_visible(False)
ax[14].set_visible(False)
plt.show()


In [ ]:
# boxplot for numerical features

fig, axs = plt.subplots(nrows = 5, ncols = 3)
ax = axs.flatten()
for i in range(len(nf)):
    df.boxplot(nf[i], ax = ax[i])
plt.tight_layout()
ax[13].set_visible(False)
ax[14].set_visible(False)
plt.show()


In [ ]:
# pairplots for all features 

g = sns.pairplot(df)
plt.title('Pairplots for all the Feature')
g.map_upper(sns.kdeplot, levels=4, color=".2")
plt.show()


In [ ]:
# compute the number of Null values in each feature

nvc = pd.DataFrame(df.isnull().sum().sort_values(), columns=['Total Null Values'])
nvc['Percentage'] = round(nvc['Total Null Values']/df.shape[0],3)*100
print(nvc)


In [ ]:
# review a few rows of the data

df.head()


In [ ]:
# replace the null values in a few features by the most frequent value in that column. 

df['Raceethnicity'].fillna(df['Raceethnicity'].mode()[0],inplace=True)
df['Sex'].fillna(df['Sex'].mode()[0],inplace=True)
df['Dentatestatus'].fillna(df['Dentatestatus'].mode()[0],inplace=True)


In [ ]:
# replace the null values in a some features by mean value.  

col_to_be_imputed = ['AgeatinterviewScreener', 'PovertyIncomeRatiounimputedincome', 'Pulseratebeatsminage5years','OverallaverageK1systolicBPage5', 'OverallaverageK5diastolicBPage5', 'Bodymassindex','Upperquadrantperiodontalassessments','Lowerquadrantperiodontalassessments','SumofpermanentDMFSduetodisease','SumofpermanentDFMSduetoanycause','SumofpermanentDFMSduetoanycause','SumofpermanentDFS','SumofpermanentDMFTduetodisease','SumofpermanentDMFTduetoanycause','Bleeding_Percentage','Furcation_SUM', 'MAL']
for item in col_to_be_imputed:
    df[item].fillna(df[item].mean(),inplace=True)


In [ ]:
# encoder categrical features by LabelEncoder. 

le = LabelEncoder()
df['Raceethnicity'] = le.fit_transform(df['Raceethnicity'])
df['Sex'] = le.fit_transform(df['Sex'])
df['Dentatestatus'] = le.fit_transform(df['Dentatestatus'])

# convert other features values from float to int

df['AgeatinterviewScreener'] = df['AgeatinterviewScreener'].astype(int)
df['PovertyIncomeRatiounimputedincome'] = df['PovertyIncomeRatiounimputedincome'].astype(int)
df['Pulseratebeatsminage5years'] = df['Pulseratebeatsminage5years'].astype(int)
df['OverallaverageK1systolicBPage5'] = df['OverallaverageK1systolicBPage5'].astype(int)
df['OverallaverageK5diastolicBPage5'] = df['OverallaverageK5diastolicBPage5'].astype(int)
df['Bodymassindex'] = df['Bodymassindex'].astype(int)
df['Upperquadrantperiodontalassessments'] = df['Upperquadrantperiodontalassessments'].astype(int)
df['Lowerquadrantperiodontalassessments'] = df['Lowerquadrantperiodontalassessments'].astype(int)
df['SumofpermanentDMFSduetodisease'] = df['SumofpermanentDMFSduetodisease'].astype(int)
df['SumofpermanentDFMSduetoanycause'] = df['SumofpermanentDFMSduetoanycause'].astype(int)
df['SumofpermanentDFMSduetoanycause'] = df['SumofpermanentDFMSduetoanycause'].astype(int)
df['SumofpermanentDFS'] = df['SumofpermanentDFS'].astype(int)
df['SumofpermanentDMFTduetodisease'] = df['SumofpermanentDMFTduetodisease'].astype(int)
df['SumofpermanentDMFTduetoanycause'] = df['SumofpermanentDMFTduetoanycause'].astype(int)
df['Bleeding_Percentage'] = df['Bleeding_Percentage'].astype(int)
df['Furcation_SUM'] = df['Furcation_SUM'].astype(int)
df['MAL'] = df['MAL'].astype(int)


In [ ]:
# display information of all features. 

df.info()


In [ ]:
# remove some useless features. 

df_feat = df.drop(['Raceethnicity','Sex','Dentatestatus','COPD_Diagnose_ja_nein'],axis=1)

# get labels for each sample.

target =  pd.get_dummies(df[['COPD_Diagnose_ja_nein']],drop_first=True)


In [ ]:
# get labels for each categrical feature. 

dummies = pd.get_dummies(df[['Raceethnicity','Sex','Dentatestatus', ]],drop_first=True)


In [ ]:
# display a few rows of the useful features. 

df_feat.head()


In [ ]:
# standardize all features data by sklearn StandardScaler

scaler = StandardScaler()
scaler.fit(df_feat)
df_scaled = scaler.transform(df_feat)
df_scaled = pd.DataFrame(df_scaled,columns=df_feat.columns[:19])
df_preprocessed = pd.concat([df_scaled,dummies,target], axis=1)
df_preprocessed.head()


In [ ]:
# Show number of features and number of samples

print('\n\033[1mInference:\033[0m The Datset consists of {} features & {} samples.'.format(df_preprocessed.shape[1], df_preprocessed.shape[0]))


In [ ]:
# see summarize of feature values

df_preprocessed.describe()


In [ ]:
# set y (values to be predicted), and X (feature values)

y = df_preprocessed['COPD_Diagnose_ja_nein']
X = df_preprocessed.drop('COPD_Diagnose_ja_nein', axis = 1)

print("y - Labels", y.shape)
print("X - N Label N id ", X.shape)
print(X.columns)


In [ ]:
# review the shapes of data frames

print(df_preprocessed.shape)
print(X.shape)
print(y.shape)


In [ ]:
# separate data to Tain or Test.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
print ('X_train: ', X_train.shape)
print ('X_test: ', X_test.shape)
print ('y_train: ', y_train.shape)
print ('y_test: ', y_test.shape)


In [ ]:
# check a few of y values

y.head()


In [ ]:
# perform logistic regression, RandomForest, SGD classify, KNN, Decision tree, Gaussian Naive Bayes, and SVM by 5-fold cross validation and show accuracy

seed = 42
scoring = 'roc_auc' 

Mymodels = []
Mymodels.append(('LogReg', LogisticRegression()))
Mymodels.append(('RandomForestClassifier', RandomForestClassifier()))
Mymodels.append(('SGDclassifier', SGDClassifier()))
Mymodels.append(('KNearestNeighbors', KNeighborsClassifier()))
Mymodels.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
Mymodels.append(('GaussianNB', GaussianNB()))
Mymodels.append(('SVM', SVC()))

results = []
names = []
for name, model in Mymodels:
    kfold = model_selection.KFold(n_splits=5, random_state=seed, shuffle=True)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) 


In [ ]:
# evaluate RandomForest model by multiple hyper parameters. 

model = RandomForestClassifier()

#param_grid = [{},]
Param_grid = {'n_estimators':[100, 200, 300], 'max_depth':[None, 5, 10], 'max_features':['auto', 'sqrt', 'log2']}

grid_search = GridSearchCV(model, param_grid = Param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)


In [ ]:
# define a random forest model

model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [ ]:
# fit the random forest model and extract feature importance.

trainFinalFI = X
yFinalFI = y

model.fit(trainFinalFI,yFinalFI)

FI_model = pd.DataFrame({"Feature Importance":model.feature_importances_,}, index=trainFinalFI.columns)
FI_model[FI_model["Feature Importance"] > 0.005].sort_values("Feature Importance").plot(kind="barh",figsize=(15,25))
plt.xticks(rotation=90)
plt.xticks(rotation=90)
plt.show()


In [ ]:
# print feature importance values. 

FI_model = pd.DataFrame({"Feature Importance":model.feature_importances_,}, index=trainFinalFI.columns)
FI_model=FI_model.sort_values('Feature Importance', ascending = False)
print(FI_model[FI_model["Feature Importance"] > 0.001])


In [ ]:
# define a function for plotting learning curve. 

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Error")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = 1-np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = 1-np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [ ]:
# plot the learning curve for the random forest model

title = "Learning Curves "
cv = ShuffleSplit(n_splits=5, test_size=0.2)
plot_learning_curve(model, title, X_train, y_train, cv=cv, n_jobs=4)
plot_learning_curve(model, title, X, y, ylim=(0.01, 0.99), cv=cv, n_jobs=4)


In [ ]:
# prepare train data and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print ('X_train: ', X_train.shape)
print ('X_test: ', X_test.shape)
print ('y_train: ', y_train.shape)
print ('y_test: ', y_test.shape)


In [ ]:
# apply the random forest model and compute confusion matrix. 

model.fit(X_train, y_train)
final_predictions = model.predict(X_test)

conf_mx = confusion_matrix(y_test, final_predictions)
print('conf_mx ready')


In [ ]:
# define a function for plot confusion matrix

def plot_confusion_matrix(cm,target_names,title='Confusion Matrix',cmap=None,
                          normalize=False):
    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


In [ ]:
# plot the confusion matrix

plot_confusion_matrix(conf_mx, 
                      normalize    = False,
                      target_names = [0,1],
                      title        = "Confusion Matrix")


In [ ]:
# compute more parameters, such as accuracy, recall, precision, F1Score.

NumClasses = 2

TP = 0
TN = 0
FP = 0
FN = 0

for z in range(NumClasses):
# One class at a time - calculate confusion matrix
    SumCM = np.sum(conf_mx)
    TPz = conf_mx[z,z]
    FNz = np.sum(conf_mx[z,:], axis=0) -TPz
    FPz = np.sum(conf_mx[:,z], axis=0) -TPz
    TNz = SumCM - (TPz+FNz+FPz)
    #FPz = np.sum(conf_mx[z], axis=-1) 
    #FPz = sum(conf_mx(:, z))-conf_mx(z, z)
    #FNz = sum(conf_mx(x, :), 2)-conf_mx(x, x)
    print('Class ',z)
  

    # Create conf matrix for class z
    cmZ = np.zeros([2, 2], dtype=np.int32)
    cmZ[0,0] = TNz
    cmZ[0,1] = FPz
    cmZ[1,0] = FNz
    cmZ[1,1] = TPz

    plot_confusion_matrix(cmZ, 
                          normalize    = False,
                          target_names = [0,1],
                          title        = "Confusion matrix for one class ")

    accuracy = (TPz+TNz)/(TPz+TNz+FPz+FNz)
    recall = TPz/(TPz+FNz)
    precision = TPz/(TPz+FPz)
    f1score = 2*recall*precision/(recall+precision)
    #roc_auc = auc(FPz, TPz)
    
    print('TPz ',TPz)
    print('FNz ',FNz)
    print('FPz ',FPz)
    print('TNz ',TNz)
    print('sum ', TPz+TNz+FPz+FNz)
    print(cmZ)
    print('Sum of CM ', np.sum(cmZ))
    print ('accuracy ',round(accuracy,4))
    print('recall ', round(recall,4))
    print('precision ', round(precision,4))
    print('F1Score ', round(f1score,4))
    print('-'*40)
    
    TP = TP + TPz
    TN = TN + TNz
    FP = FP + FPz
    FN = FN + FNz
print ('TN: ', TN)
print ('FP: ', FP)
print ('FN: ', FN)
print ('TP: ', TP)
print('_'*40)   


**NN model**  

In [ ]:
# prepare train data, test data for neural network model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print ('X_train: ', X_train.shape)
print ('X_test: ', X_test.shape)
print ('y_train: ', y_train.shape)
print ('y_test: ', y_test.shape)

print(y_train)
print(y_test)


In [ ]:

x_val = X_test
partial_x_train = X_train
y_val = y_test
partial_y_train = y_train

print("partial_x_train ", partial_x_train.shape)
print("partial_y_train ", partial_y_train.shape)

print("x_val ", x_val.shape)
print("y_val ", y_val.shape)


In [ ]:

yTrain = to_categorical(partial_y_train)
yVal = to_categorical(y_val)
print(yTrain.shape)
print(yVal.shape)


In [ ]:
# build neural network model

model = models.Sequential()
model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(19,)))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
print(model.summary())

# FIT/TRAIN model

NumEpochs = 20
BatchSize = 128
lr=1e-4
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['categorical_accuracy', tf.keras.metrics.AUC(name='auc')])

history = model.fit(partial_x_train, yTrain, epochs=NumEpochs, batch_size=BatchSize, validation_data=(x_val, yVal))

results = model.evaluate(x_val, yVal)
print("_"*100)
print("Test Loss and Accuracy")
print("results ", results)
history_dict = history.history
history_dict.keys()


In [ ]:
# evaluate the model by validation data

evalu = model.evaluate(x_val, yVal)
print(model.metrics_names)
print(evalu)


In [ ]:
# plot VALIDATION LOSS curves

plt.clf()
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, (len(history_dict['loss']) + 1))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# plot VALIDATION ACCURACY curves

plt.clf()
acc_values = history_dict['categorical_accuracy']
val_acc_values = history_dict['val_categorical_accuracy']
epochs = range(1, (len(history_dict['categorical_accuracy']) + 1))
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Final Fit / Predict

final_predictions = model.predict(x_val)
print(final_predictions)


In [ ]:
# PREDICT & ARGMAX to get the digit from the probability of softmax layer

print(final_predictions)
pred = []
numTest = final_predictions.shape[0]
for i in range(numTest):
    pred.append(np.argmax(final_predictions[i])) 
predictions = np.array(pred)  
print(predictions)


In [ ]:
# PREDICT & ARGMAX to get the digit from the probability of softmax layer

print(yVal)
pred = []
numTest = yVal.shape[0]
for i in range(numTest):
    pred.append(np.argmax(yVal[i])) 
yValNum = np.array(pred)  
print(yValNum)


In [ ]:
# compute confusion_matrix

conf_mx = confusion_matrix(yValNum, predictions)
print('conf_mx ready')


In [ ]:
# plot confusion matrix

plot_confusion_matrix(conf_mx, 
                      normalize    = False,
                      target_names = [0,1],
                      title        = "Confusion Matrix")


In [ ]:
# compute AUC ROC

from sklearn.metrics import roc_auc_score
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
        lb = LabelBinarizer()
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)
        return roc_auc_score(y_test, y_pred, average=average)

print('AUC ROC ',multiclass_roc_auc_score(yValNum, predictions))
